In [2]:
# NHSRC PHC SUPPLY CHAIN - ACTION RULES & ESCALATION LOGIC
import pandas as pd
import numpy as np
import json
import warnings
warnings.filterwarnings('ignore')

print("🚨 NHSRC PHC ACTION RULES & ESCALATION LOGIC")
print("=" * 70)

# 1️⃣ LOAD FINAL DECISION MATRIX
print("📥 1. Loading Final Decision Matrix...")

# Load final decision matrix (Day 8 output)
df = pd.read_csv("reports/final_decision_matrix.csv")

# Load forecast confidence from previous reports
try:
    confidence_data = pd.read_csv("reports/best_model_selection.csv")
    confidence_data = confidence_data[['sku_id', 'forecast_confidence']]
    df = pd.merge(df, confidence_data, on='sku_id', how='left')
except:
    print("   ⚠️  Forecast confidence data not found, using resilience band as proxy")
    df['forecast_confidence'] = df['resilience_band'].apply(
        lambda x: 'HIGH' if x == 'Stable' else 'MEDIUM' if x == 'Watchlist' else 'LOW'
    )

print(f"   Loaded {len(df)} SKUs")
print(f"   Priority distribution:")
for priority in sorted(df['final_action_priority'].unique()):
    count = (df['final_action_priority'] == priority).sum()
    print(f"   - Priority {priority}: {count} SKUs")

# 2️⃣ ADD ESCALATION LAYER
print("\n👥 2. Adding Escalation Layer...")

def assign_role(row):
    p = row["final_action_priority"]
    if p == 1:
        return "Escalate to District SCM + Medical Officer"
    if p == 2:
        return "PHC Pharmacist Action Required"
    if p == 3:
        return "Review in Weekly Procurement Meeting"
    if p == 4:
        return "Monitor Only"
    return "Archive - No Action Required"

df["responsible_person"] = df.apply(assign_role, axis=1)

print("   Responsible Person Distribution:")
role_counts = df["responsible_person"].value_counts()
for role, count in role_counts.items():
    percentage = (count / len(df)) * 100
    print(f"   - {role}: {count} SKUs ({percentage:.1f}%)")

# 3️⃣ ADD ACTION WINDOW (OPERATIONAL DEADLINE)
print("\n⏰ 3. Adding Action Window...")

def action_window(p):
    return {
        1: "Within 24 Hours",
        2: "Within 3 Days",
        3: "Within 7 Days",
        4: "Next Review Cycle (15 days)",
        5: "No Action Required"
    }[p]

df["action_window"] = df["final_action_priority"].apply(action_window)

print("   Action Window Distribution:")
window_counts = df["action_window"].value_counts()
for window, count in window_counts.items():
    print(f"   - {window}: {count} SKUs")

# 4️⃣ ADD CONFIDENCE-BASED MESSAGE SEVERITY
print("\n📊 4. Adding Message Severity Based on Confidence...")

df["message_style"] = df.apply(
    lambda r: "Hard Alert" if r["resilience_band"] in ["Critical", "Risky"] 
              and r["forecast_confidence"] == "HIGH" else "Soft Advisory",
    axis=1
)

print("   Message Style Distribution:")
style_counts = df["message_style"].value_counts()
for style, count in style_counts.items():
    percentage = (count / len(df)) * 100
    print(f"   - {style}: {count} SKUs ({percentage:.1f}%)")

# 5️⃣ GENERATE HUMAN-READABLE ACTION MESSAGES
print("\n💬 5. Generating Human-Readable Action Messages...")

def generate_message(row):
    # Determine emoji based on priority
    if row['final_action_priority'] == 1:
        emoji = "🚨"
    elif row['final_action_priority'] <= 3:
        emoji = "⚠️"
    else:
        emoji = "ℹ️"
    
    # Create concise reason
    if row['cover_gap_vs_min'] < 0:
        reason = f"Stock coverage {row['days_cover']:.1f}d < minimum requirement {row['min_days_cover']}d."
    elif row['resilience_band'] == 'Critical':
        reason = f"Critical resilience score ({row['resilience_score']:.0f}/100) with stockout risk."
    else:
        reason = f"Stock coverage {row['days_cover']:.1f}d requires monitoring."
    
    # Add specific guidance based on action
    if 'EMERGENCY' in row['final_action']:
        guidance = "Raise indent in DVDMS → Allocate buffer stock → Enable FEFO redistribution if available nearby."
    elif 'REDISTRIBUTE' in row['final_action']:
        guidance = "Check nearby PHCs for stock transfer → Update batch expiry tracking → Prioritize in daily dispensing."
    elif 'REORDER' in row['final_action']:
        guidance = "Prepare purchase requisition → Check supplier lead time → Update stock position in LMIS."
    else:
        guidance = "Continue regular monitoring → Update stock cards → Report in weekly review."
    
    return (
        f"{emoji} {row['sku_id']} | {row['sku_name']} | {row['ved_category']} | {row['fsn_category']}\n"
        f"Action: {row['final_action']}\n"
        f"Reason: {reason}\n"
        f"Escalation: {row['responsible_person']}\n"
        f"Deadline: {row['action_window']}\n"
        f"Next Step: {guidance}\n"
        f"Confidence: {row['forecast_confidence']}\n"
        f"{'-'*50}"
    )

df["action_message"] = df.apply(generate_message, axis=1)

print("   Generated actionable messages for all SKUs")

# 6️⃣ SAVE FINAL ACTIONS WITH ESCALATION
print("\n💾 6. Saving Final Actions with Escalation...")

# Save comprehensive CSV
output_path_csv = "reports/final_actions_with_escalation.csv"
df.to_csv(output_path_csv, index=False)
print(f"   ✅ {output_path_csv}")

# 7️⃣ CREATE SYSTEM ALERT JSON FOR DASHBOARD INTEGRATION
print("\n📱 7. Creating System Alert JSON for Dashboard Integration...")

# Filter for high-priority alerts (Priority 1-2)
alerts = df[df["final_action_priority"] <= 2][[
    "sku_id", "sku_name", "final_action", "responsible_person", 
    "action_window", "resilience_band", "forecast_confidence",
    "days_cover", "min_days_cover", "cover_gap_vs_min"
]].copy()

# Add timestamp and alert ID
alerts['alert_id'] = [f"ALERT_{i:04d}" for i in range(1, len(alerts) + 1)]
alerts['timestamp'] = pd.Timestamp.now().isoformat()
alerts['severity'] = alerts['resilience_band'].apply(
    lambda x: 'CRITICAL' if x == 'Critical' else 'HIGH'
)

# Convert to dictionary
alerts_dict = alerts.to_dict(orient="records")

# Save as JSON
output_path_json = "reports/actionable_alerts.json"
with open(output_path_json, "w") as f:
    json.dump(alerts_dict, f, indent=2)

print(f"   ✅ {output_path_json} ({len(alerts_dict)} alerts generated)")

# 8️⃣ CREATE SUMMARY FOR PHC MANAGER
print("\n📋 8. Creating PHC Manager Summary...")

# Count urgent actions
urgent_count = (df['final_action_priority'] <= 2).sum()
critical_count = (df['resilience_band'] == 'Critical').sum()

print(f"   ⚠️  URGENT ACTIONS REQUIRED: {urgent_count} of {len(df)} SKUs")
print(f"   🚨 CRITICAL STATUS: {critical_count} SKUs")

# List critical items
critical_items = df[df['resilience_band'] == 'Critical'][['sku_id', 'sku_name', 'final_action']]
if len(critical_items) > 0:
    print("\n   CRITICAL ITEMS NEEDING IMMEDIATE ATTENTION:")
    for _, item in critical_items.iterrows():
        print(f"   - {item['sku_id']}: {item['sku_name']} → {item['final_action']}")

# 9️⃣ FINAL OUTPUTS FOR TRAINER
print("\n" + "="*70)
print("🎯 TRAINER OUTPUTS")
print("="*70)

print("\n1. 🔹 FIRST 5 MESSAGES FROM ACTION_MESSAGE COLUMN:")
print("-" * 70)
for i, message in enumerate(df['action_message'].head(5), 1):
    print(f"\nMessage {i}:")
    print(message)

print("\n2. 🔹 COUNT OF ALERTS BY RESPONSIBLE_PERSON:")
print("-" * 70)
for role, count in role_counts.items():
    print(f"   {role}: {count} SKUs")

print("\n3. 🔹 COUNT OF MESSAGE_STYLE:")
print("-" * 70)
for style, count in style_counts.items():
    print(f"   {style}: {count} SKUs")

print("\n4. 🔹 UPDATED GIT LS-FILES:")
print("-" * 70)
import subprocess
result = subprocess.run(['git', 'ls-files'], capture_output=True, text=True)
print(result.stdout)

print("\n" + "="*70)
print("✅ DAY 9 ACTION RULES & ESCALATION LOGIC COMPLETE")
print("="*70)
print("\n📌 OPERATIONAL TRANSFORMATION ACHIEVED:")
print("   • Mathematical predictions → Actionable instructions")
print("   • Clear responsibility assignment (Who does what)")
print("   • Time-bound deadlines (By when)")
print("   • Human-readable messages for PHC staff")
print("   • Dashboard-ready JSON alerts for system integration")

🚨 NHSRC PHC ACTION RULES & ESCALATION LOGIC
📥 1. Loading Final Decision Matrix...
   Loaded 12 SKUs
   Priority distribution:
   - Priority 1: 5 SKUs
   - Priority 2: 1 SKUs
   - Priority 3: 1 SKUs
   - Priority 4: 4 SKUs
   - Priority 5: 1 SKUs

👥 2. Adding Escalation Layer...
   Responsible Person Distribution:
   - Escalate to District SCM + Medical Officer: 5 SKUs (41.7%)
   - Monitor Only: 4 SKUs (33.3%)
   - PHC Pharmacist Action Required: 1 SKUs (8.3%)
   - Review in Weekly Procurement Meeting: 1 SKUs (8.3%)
   - Archive - No Action Required: 1 SKUs (8.3%)

⏰ 3. Adding Action Window...
   Action Window Distribution:
   - Within 24 Hours: 5 SKUs
   - Next Review Cycle (15 days): 4 SKUs
   - Within 3 Days: 1 SKUs
   - Within 7 Days: 1 SKUs
   - No Action Required: 1 SKUs

📊 4. Adding Message Severity Based on Confidence...
   Message Style Distribution:
   - Hard Alert: 7 SKUs (58.3%)
   - Soft Advisory: 5 SKUs (41.7%)

💬 5. Generating Human-Readable Action Messages...
   Generate